# Explanaition
This script is:
0. uses your Settings defined below for the next steps
1. creates a list of commands and saves them in a commands file, 
   1. so every session can be run in an individual job
   2. and every job could theoretically run at the same time
2. creates a SLURM jobs script based on your
   1. that reads the commands list and creates a job for every line in the commands file
3. runs the created SLURM script
   1. Outputs/Errors are saved in AnimalClass/Scicore/Outputs

## Folder Structure

### Femtonics (Mesc Files)

```
───DON-019608
    │   DON-019608.yaml
    │   
    ├───20240126
    │   │   20240126.yaml
    │   │   
    │   └───002P-F
    │             DON-019608_20240126_002P-F_S1-S2-ACQ.mesc
    │
    └───20240127
        │   ...
       ...
```

### Bscope (RAW Files)

```
───DON-019608
    │   DON-019608.yaml
    │   
    ├───20240126
    │   │   20240126.yaml
    │   │   
    │   └───data
    │           Image_001_001.raw
    │
    └───20240127
        │   ...
       ...
```

## Imports

In [2]:
# Imports
from create_commands_list import create_commands_file
from scicore_file_creation import create_sbatch_file, create_commands_file, run_pipeline
%matplotlib inline
#%matplotlib tk
%autosave 180
%load_ext autoreload
%autoreload 2
%config Completer.use_jedi = False

Autosaving every 180 seconds


## Settings

In [11]:
# Settings

## Names
username = "mauser00"                   # Your username
donafl00_user_folder_name = "Sergej"    # Your \\toucan-all.scicore.unibas.ch\donafl00-calcium$\Users\
project_folder = "Intrinsic_Imaging"    # Your project folder
conda_env_name = "animal_env"           # Write your definded your conda environment
mail_user = "your.name@unibas.ch"       # Your mail address for notifications
pipeline_name = "pipeline"              # .sh script naming: Choose as you want
commands_fname = "commands.cmd"         # commands file_name used in .sh script: Choose as you want
job_name = "converting"                 # job_name while running on scicore: Choose as you want
output_name = "converted"               # file_name in output folder: Choose as you want


## Pipeline Settings
mesc_to_tiff=True           # estimated time 1h
suite2p=True                # estimated time 2h
binarize=True               # estimated time 0.1h
pairwise_correlate=False    # estimated time 2h


## Paths
## running on Scicore
project_root_dir = f"/scicore/projects/donafl00-calcium/Users/{donafl00_user_folder_name}/{project_folder}"  # Your project root directory


## Dataset choosing (choose all by writing ["all"])
wanted_animal_ids = ["DON-019608"]  # Choose the animal ids you want to process, e.g. ["DON-019608", "DON-019609"]
wanted_session_ids = ["20240126"]  # Choose the session ids you want to process, e.g. ["20240126", "20240127"]


## Harware
### Number of cores and RAM per cpu
n_cpus = 10
memory = 8


## Time reservation for each individal session on scicore
days = 0
hours = 6
minutes = 0
seconds = 0

## Running

In [14]:
# create list of commands for running multiple jobs at once
commands_fname = create_commands_file(
    commands_fname=commands_fname,
    wanted_animal_ids=wanted_animal_ids,
    wanted_session_ids=wanted_session_ids,
    project_root_dir=project_root_dir,
    mesc_to_tiff=mesc_to_tiff,
    suite2p=suite2p,
    binarize=binarize,
    pairwise_correlate=pairwise_correlate,
)

Added animal: DON-019608
Initialized session: DON-019608 20240126


In [9]:
# create jobscript for running every command in the commands file
sbatch_fname = create_sbatch_file(
    commands_fname = commands_fname,
    pipeline = pipeline_name,
    job_name = job_name,
    n_cpus = n_cpus,
    memory_per_cpu = memory,
    days = days,
    n_hours = hours,
    n_minutes = minutes,
    n_seconds = seconds,
    start_line = None, # can be set to skip the first n lines of the commands file
    end_line = None, # can be set to stop after the first n lines of the commands file
    output_name = output_name,
    mail_user = mail_user,
    conda_env_name = conda_env_name,
    username = username,
)

In [ ]:
# run jobscript
run_pipeline(sbatch_fname)

## Usefull Terminal Slurm Commands

1. Check in the Terminal if the jobs are running
``` bash
squeue -u <username> -s
```
2. Check every 60 seconds if the pipeline is running
```bash
   while true;do squeue -u <username> -s; sleep 60; done
```
3. Cancel running job
```bash
    # by Job ID
    scancel <jobID>
    # by username
    scancel -u <username>
```